In [68]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from python_module.pricing_model import BlackScholesModel, SABRModel

pd.options.display.float_format = '{:.2f}'.format

In [69]:
def compute_position_sizing(constraints, slide_df, expected_pnl, l1_regularization):

    def objective_function(weights, expected_pnl, l1_regularization):
        return - expected_pnl.dot(weights) + (np.abs(weights).sum() * l1_regularization)

    bnds = [(-100, 100)] * len(expected_pnl.index)

    x0 = [0] * len(expected_pnl.index)

    result = minimize(objective_function, x0, method='SLSQP', bounds=bnds,constraints=constraints, args=(expected_pnl, l1_regularization))

    summary = pd.concat([slide_df, expected_pnl], axis=1)
    summary['weights'] = result.x

    portfolio_slide = slide_df.multiply(result.x, axis=0).sum()
    portfolio_expected_pnl = expected_pnl.dot(result.x)

    print('portfolio slide')
    display(portfolio_slide)

    print(f'portfolio expected pnl {portfolio_expected_pnl:.2f}')

    return summary, result.x

In [70]:
# -------
# GENERATE MARKET DATA AND GREEKS
# -------

# Explicit pricing parameters
S0 = F = 100
r = 0
option_type = 'put'
time_to_maturity = 250

# Market parameters
market_alpha = +0.1
market_beta  = +1.0
market_rho   = -0.5
market_nu    = 1.0

# Realized parameters
realized_alpha = +0.1
realized_beta  = +1.0
realized_rho   = -0.5
realized_nu    = 0.5
# Pre-processing
T = time_to_maturity / 250

# Pricing factory
market_data_list = list()
realized_data_list = list()
for K in np.linspace(start=80, stop=120, num=20, dtype=int):

    option_type = 'call' if K > 100 else 'put'
    
    market_pricing_results = SABRModel.compute_option(F, K, T, market_alpha, market_beta, market_rho, market_nu, r, option_type, slide_list=[-0.05, 0.05])
    realized_pricing_results = SABRModel.compute_option(F, K, T, realized_alpha, realized_beta, realized_rho, realized_nu, r, option_type, slide_list=[-0.05, 0.05])
    
    market_data_list.append({
        'symbol': f"{time_to_maturity}_{K}_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': time_to_maturity, 
        'S0':S0, 'r': r, 'F': F, 'K': K, 'T': T, **market_pricing_results})

    realized_data_list.append({
        'symbol': f"{time_to_maturity}_{K}_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': time_to_maturity, 
        'S0':S0, 'r': r, 'F': F, 'K': K, 'T': T, **realized_pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')

realized_data_df = pd.DataFrame(realized_data_list)
realized_data_df = realized_data_df.set_index('symbol')

In [71]:
# -------
# SLIDE & EXPECTED PNL
# -------
slide_df = market_data_df[list(filter(lambda x: x.startswith('slide'), market_data_df.columns))]
expected_pnl = realized_data_df['price']-market_data_df['price']
expected_pnl.name = 'expected_pnl'

In [72]:
cons = [
    {'type': 'ineq', 'fun': lambda weights: slide_df.iloc[:, 1].dot(weights)},
    {'type': 'ineq', 'fun': lambda weights: slide_df.iloc[:, 0].dot(weights)}
]
summary, weights = compute_position_sizing(cons, slide_df, expected_pnl, l1_regularization=0.)
summary

portfolio slide


slide pnl -0.05    0.00
slide pnl 0.05    -0.00
dtype: float64

portfolio expected pnl 471.13


,slide pnl -0.05,slide pnl 0.05,expected_pnl,weights
symbol,,,,
250_80_put,0.31,-0.01,-0.52,-100.00
250_82_put,0.35,-0.01,-0.55,-100.00
250_84_put,0.39,0.00,-0.58,-100.00
250_86_put,0.44,0.02,-0.59,-100.00
250_88_put,0.50,0.03,-0.59,-100.00
250_90_put,0.57,0.05,-0.57,-100.00
250_92_put,0.65,0.07,-0.52,-100.00
250_94_put,0.74,0.09,-0.45,-100.00
250_96_put,0.84,0.10,-0.35,-60.56
